In [2]:
from pydantic import BaseModel
from typing import Literal, Optional, List

import json
import ollama

In [3]:
from pydantic import BaseModel
from typing import List

class EvaluatedQA(BaseModel):
    question: str
    answered: Literal['yes', 'partially', 'no']
    answer_summary: str = None
    answer_quote: str = None

class InterventionAnalysis(BaseModel):
    intervention: str
    response: str
    evaluations: List[EvaluatedQA]


In [ ]:

class QAExtractor:
    def __init__(self, model_name: str = "llama3"):
        self.model = model_name

    def extract_qa(self, intervention: str, response: str) -> InterventionAnalysis:
        messages = [
            {
                "role": "system",
                "content": """
                              You are an assistant that analyzes multi-question interventions from business or financial conference calls.

                              Given an intervention that may contain several questions, and the subsequent response:
                              - Extract each individual question clearly.
                              - Determine whether each question is answered ("yes"), partially answered ("partially"), or not answered ("no").
                              - If it is answered (fully or partially), provide a short summary and a quote from the response that supports your evaluation.
                              """
                                          },
                                          {
                "role": "user",
                "content": f"""
                              Intervention:
                              {intervention}

                              Response:
                              {response}
                """
            }
        ]

        result = json.loads(ollama.chat(model=self.model, messages=messages, format=InterventionAnalysis.model_json_schema()).message.content)
        return result

In [10]:
extractor = QAExtractor(model_name="llama3")

with open("/home/aacastro/Alejandro/ACA_MultichanelAI_2025/src/TEXT/POO/annotated/AAPL/2018/Q3.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Ahora puedes acceder a los pares por clave
for pair_key, pair_content in data.items():
    question = pair_content["Question"]
    answer = pair_content["Answer"]
    # print(f"{pair_key}:\nQ: {question}\nA: {answer}\n")
    qa_results = extractor.extract_qa(question, answer)
    print(qa_results['evaluations'])

[{'question': "First for Tim, you're on track to hit your services revenue target even earlier than planned. So how are you thinking about the next legs of services growth as you move into the next three to five years?", 'answered': 'yes', 'answer_summary': 'The company is thrilled with their current services results and feels great about their pipeline for new services.', 'answer_quote': "We feel great about our current services, but obviously we're also thrilled about our pipeline that have some new services in it as well."}, {'question': "And then for you, Luca, NAN prices are falling this year. Services mix is rising. Those should both positively influence gross margins and yet We're seeing gross margins sort of hang out here at 38%. What are the offsetting headwinds, and is it possible that we could see the tailwinds start to overpower those headwinds in the next couple of quarters and see gross margins drift higher?", 'answered': 'partially', 'answer_summary': "The company mentio